# Astra target landing site example

In [ ]:
%load_ext autoreload
%autoreload
import astra
from astra.target_landing import targetFlight
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
from scipy.interpolate import interp1d
from astra.weather import forecastEnvironment
from astra.GFS import GFS_Handler
import astra
import os
%matplotlib notebook



In [ ]:
targetLat = 28.33
targetLon = -79.8
targetElev = 0.

# Use a previous forecast (allows offline development)
launch_datetime = datetime(2017, 4, 24,hour=12,minute=15)
simEnvironment = forecastEnvironment(launchSiteLat=29.2108,      # deg
                                     launchSiteLon=-81.0228,     # deg
                                     launchSiteElev=4,           # m
                                     dateAndTime=launch_datetime,
                                     forceNonHD=True,
                                     debugging=True)
# # Set up the example input data files (from 24/04/2017, Daytona Beach)
# fileDict = {}
# for paramName in GFS_Handler.weatherParameters.keys():
#     fileDict[paramName] = os.path.join(os.path.dirname(astra.__file__),
#         '../test/example_data',
#         'gfs_0p50_06z.ascii?{}[12:15][0:46][231:245][545:571]'.format(paramName))
# simEnvironment.loadFromNOAAFiles(fileDict)
# inputs = {'launchSiteForecasts': [simEnvironment]}

simulator = targetFlight(start_dateTime=launch_datetime,
                 targetLat=targetLat,
                 targetLon=targetLon,
                 targetElev=targetElev,
                 launchSites=[(29.2108, -81.0228, 4.)],
                 balloonModel='TA100',
                 balloonGasType="Helium",
                 nozzleLift=1,
                 trainEquivSphereDiam=0.1,
                 inflationTemperature=0.0,
                 payloadTrainWeight=0.38,
                 windowDuration=24,
                 HD=False,
                 maxFlightTime=18000,
                 parachuteModel=None,
                 debugging=False,
                 log_to_file=False,
                 progress_to_file=False,
                 outputFile=os.path.join(''))

bestProfile = simulator.bruteForce()

In [ ]:
fig1, ax1 = simulator.plotPaths3D()

fig2, ax2 = simulator.plotLandingSites()

# add the point obtained by scipy:
# best = simulator.optimizeTargetLandingSite(method='Nelder-Mead', x0=0.5)



In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)


plt.axis('equal')
dateTimes = []
distances = []
for prof in simulator.results:
    dateTime = prof.launchDateTime
    dateTimes.append(dateTime)
    distance = prof.distanceFromTarget
    distances.append(distance)

ax.plot(dateTimes, distances, 'bx', label='Trajectory simulations')
ax.plot(simulator.bestProfile.launchDateTime, simulator.bestProfile.distanceFromTarget, 'k*', label='Brute Force min')

import matplotlib.dates as mdates

xtick_locator = mdates.AutoDateLocator()
xtick_formatter = mdates.AutoDateFormatter(xtick_locator)
ax.xaxis.set_major_locator(xtick_locator)
ax.xaxis.set_major_formatter(xtick_formatter)
fig.autofmt_xdate()

ax.set_xlabel('Date and Time')
ax.set_ylabel(r'$|\bf{X} - \bf{X_{target}}|$ (deg)')

In [ ]:
# test the scipy algorithms:

datetimeVector =  np.linspace(0, simulator.windowDuration, len(distances_map.keys())) 
distanceVector = list(distances_map.values())

f_curve = interp1d(datetimeVector, distanceVector, kind='cubic')

new_timedelta_vector = np.linspace(0, simulator.windowDuration, 100)
newdistances = f_curve(new_timedelta_vector)
new_datetime_vector = [simulator.start_dateTime + timedelta(hours=h) for h in new_timedelta_vector]

ax.plot(new_datetime_vector, newdistances, label='Fit (cubic)')

In [ ]:
import scipy.optimize

res = scipy.optimize.minimize(f_curve, x0=[0.3*simulator.windowDuration], method='BFGS',
#                               bounds=[(0.1, simulator.windowDuration-0.1)],
                             options={"disp": True})
best_time_model = simulator.start_dateTime + timedelta(hours=res.x[0])
# res.fun
ax.plot(best_time_model, res.fun, '^r', label=r'Nelder-Mead, $x_0=0.3 \times dt$')

res = scipy.optimize.minimize(f_curve, x0=[0.7*simulator.windowDuration], method='BFGS',
#                               bounds=[(0.1, simulator.windowDuration-0.1)],
                             options={"disp": True})
best_time_model = simulator.start_dateTime + timedelta(hours=res.x[0])
# res.fun
ax.plot(best_time_model, res.fun, 'or', label=r'Nelder-Mead, $x_0=0.7 \times dt$')

In [ ]:
help(scipy.optimize.minimize)

In [ ]:
import scipy.optimize

res = scipy.optimize.differential_evolution(f_curve,
                              bounds=[(0.1, simulator.windowDuration-0.1)],
                              polish=False)
best_time_de = simulator.start_dateTime + timedelta(hours=res.x[0])
# res.fun
ax.plot(best_time_de, res.fun, '^g', label='DE')
print(res.nit)
lgd = ax.legend(loc='upper left')
ax.set_facecolor("white")
plt.tight_layout()
fig.savefig('brute_force_comparison.png', bbox_extra_artists=(lgd,), bbox_inches='tight')


## Testing

In [ ]:
 datetimeVector = [self.start_dateTime + timedelta(hours=t)
                            for t in range(self.windowDuration)]

nozzleLiftLowerBound = 0.4
nozzleLiftUpperBound = 1.0
nozzelLift_Vector = np.linspace(nozzleLiftLowerBound, nozzleLiftUpperBound)
self.results = {}

distance_map = {}
for i, launchSiteForecast in enumerate(self.launchSiteForecasts):
    self.environment = launchSiteForecast

    for j, t in enumerate(datetimeVector):
        distance_lift_vec = np.zeros(np.length(nozzelLift_Vector))
        for k, L in enumerate(nozzelLift_Vector):
            # brute force approach
            distance = self.targetDistance(t)
            distance_lift_vec[k] = distance

        distance_map[t] = distance_Vector  

In [ ]:
# ascent rate penalty function
